In [2]:
import sys
!{sys.executable} -m pip install pafy
!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

    100% |################################| 1.9MB 265kB/s eta 0:00:01


In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
import traitlets
import PIL
from IPython.display import display, Image
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
from my_heatmap_generator import HMap
import cv2

device = torch.device('cuda')
mtcnn = MTCNN(keep_all=True, device=device)
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()

print("model loaded")
outframe = None
url = "https://youtu.be/w_RxkekW7sE"
load_dotenv()

#defaults
active = "Live"
default_radius = getenv('DEFAULT_RADIUS')
default_duration = getenv('DEFAULT_DURATION')
radius = int(getenv('DEFAULT_RADIUS'))
duration = int(getenv('DEFAULT_DURATION'))

def on_value_change(change):
    print("Value change")
    radius = radius_slider.value
    duration = duration_slider.value

duration_slider = widgets.IntSlider(
    value=duration,
    min = 2,
    max = 30,
    step=1,
    description='Airborne Duration:',
    continuous_update=True,
    orientation='horizontal'
)

radius_slider = widgets.IntSlider(
    value=duration,
    min = 10,
    max = 124,
    step=1,
    description='Radius:',
    continuous_update=True,
    orientation='horizontal'
)

def reset_values(b):
    radius_slider.value = int(default_radius)
    duration_slider.value = int(default_duration)

reset_button = widgets.Button(description = "Reset")
reset_button.on_click(reset_values)

model loaded


In [ ]:
HMAP = []
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0)
camera.running = True
camera.value

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    global hmap, radius, duration
    frame = change['new']
    frame =cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(frame)
    frame = PIL.Image.fromarray(frame)
    if boxes is not None:
        for box in boxes:
            xrange = int((box[2] - box[0])/2)
            yrange = int((box[3] - box[1])/2)
            min_x = box[0] - xrange if box[0]-xrange > 0 else 0
            max_x = box[2] + xrange if box[2]+xrange < 224 else 224
            min_y = box[1] - yrange if box[1]-yrange > 0 else 0
            max_y = box[3] + yrange if box[3]+yrange < 224 else 224
            rectangle = (min_x, min_y, max_x, max_y)
            face_only = frame.crop(rectangle)
            new_width  = int(224 * face_only.size[0] / face_only.size[1])
            face_only = face_only.resize((new_width, 224))
            processed = preprocess(face_only)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            radius = radius_slider.value
            duration = duration_slider.value
            if output[category_index] < 0.6:
                category_index = 1
            hmap = HMap(box, category_index)
            HMAP.append(hmap)
            frame = hmap.draw_map(frame, radius, duration)
    for hmap in HMAP:
        frame = hmap.draw_map(frame, radius, duration)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

camera.observe(execute, names='value')
display_handle = display(image_widget, widgets.HBox([duration_slider, radius_slider, reset_button]))

In [ ]:
from threading import Thread
import pafy
import ffmpeg

HMAP = []
#urlPafy = pafy.new(url)
#link = urlPafy.getbest(preftype="mp4")
#video = cv2.VideoCapture(link.url)

filename = './file.mp4'
probe = ffmpeg.probe(filename)
vid_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
width = int(vid_info['width'])
height = int(vid_info['height'])
frame_count = int(vid_info['nb_frames'])
out, err = ffmpeg.input(filename).output('pipe:', format='rawvideo', pix_fmt='rgb24')
frames = np.frombuffer(out, np.unit8).reshape([-1, height, width, 3])

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

image_widget = widgets.Image(format='jpeg', width=400, height=400)

urlbox = widgets.Text(
    value=url,
    placeholder=url,
    disabled=False
)

def executeVid(frame):
    global hmap, radius, duration
    frame =cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(frame)
    frame = PIL.Image.fromarray(frame)
    if boxes is not None:
        for box in boxes:
            xrange = int((box[2] - box[0])/2)
            yrange = int((box[3] - box[1])/2)
            min_x = box[0] - xrange if box[0]-xrange > 0 else 0
            max_x = box[2] + xrange if box[2]+xrange < 224 else 224
            min_y = box[1] - yrange if box[1]-yrange > 0 else 0
            max_y = box[3] + yrange if box[3]+yrange < 224 else 224
            rectangle = (min_x, min_y, max_x, max_y)
            face_only = frame.crop(rectangle)
            if face_only.size[0] > 0 and face_only.size[1] > 0:
                new_width  = int(224 * face_only.size[0] / face_only.size[1])
                face_only = face_only.resize((new_width, 224))
                processed = preprocess(face_only)
                output = model(processed)
                output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
                category_index = output.argmax()
                radius = radius_slider.value
                duration = duration_slider.value
                if output[category_index] < 0.6:
                    category_index = 1
                hmap = HMap(box, category_index)
                HMAP.append(hmap)
                frame = hmap.draw_map(frame, radius, duration)
    for hmap in HMAP:
        frame = hmap.draw_map(frame, radius, duration)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()
    
def submit_url(b):
    probe = ffmpeg.probe(urlbox.value)
    vid_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    width = int(vid_info['width'])
    height = int(vid_info['height'])
    frame_count = int(vid_info['nb_frames'])
    out, err = ffmpeg.input(urlbox.value).output('pipe:', format='rawvideo', pix_fmt='rgb24')
    frames = np.frombuffer(out, np.unit8).reshape([-1, height, width, 3])
    for frame in frames:
        executeVid(frame)

submit_button = widgets.Button(description = "Submit")
submit_button.on_click(submit_url)    
display_handle = display(widgets.HBox([urlbox, submit_button]), image_widget, widgets.HBox([duration_slider, radius_slider, reset_button]))